In [1]:
library(lme4)
library(lmerTest)
library(sjPlot)
library(broom.mixed)


Loading required package: Matrix


Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step


#refugeeswelcome

Registered S3 method overwritten by 'broom.mixed':
  method      from 
  tidy.gamlss broom



# Load data

In [2]:
df <- read.csv("output/critical_long_format.csv")
#df <- read.csv("output/critical_long_format_with_dummy.csv")
df$ID <- df$client_id

# Data preparation

In [3]:
# center binary variables
df$critical_cent <- scale(df$critical, scale=F)
df$mail_cent <- scale(df$mailed, scale=F)
df$mailday_cent <- scale(df$mailday, scale=F)
df$weekday_cent <- scale(df$is_weekday, scale=F)


In [4]:
# factors for green hours, mail and mailday
df$critical_f <- as.factor(df$critical)
levels(df$critical_f) <- c("No green hours","Green hours (11am to 3pm)")
df$mail_f <- as.factor(df$mailed)
levels(df$mail_f) <- c("No mail","Green mail")
df$mail_day_f <- as.factor(df$mailday)
levels(df$mail_day_f) <- c("No mail for day","Mail for day")
df$weekday_f <- as.factor(df$is_weekday)
levels(df$weekday_f) <- c("Mo-Fr","Sa-Su")

In [5]:
df_maildays = df[df$mailday == 1,]
df_nomaildays = df[df$mailday == 0,]

df_nomail = df[df$mailed == 0,]

# Regression Tables

## A.1. Model 1 - Interaction of mail and critical time on charges and kWh

### Dependant variable:  Number of charges

In [6]:
m1 <- glmer(charge_started ~ mail_cent*critical_cent+(1|ID),data=df_maildays,family = binomial(link = "logit"))
summary(m1)
confint(m1,parm="beta_",method="Wald")
tidy(m1,conf.int=TRUE,exponentiate=TRUE,effects="fixed")


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: charge_started ~ mail_cent * critical_cent + (1 | ID)
   Data: df_maildays

     AIC      BIC   logLik deviance df.resid 
  1000.3   1035.4   -495.1    990.3     8263 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.0797 -0.0684 -0.0240 -0.0205 15.9540 

Random effects:
 Groups Name        Variance Std.Dev.
 ID     (Intercept) 6.35     2.52    
Number of obs: 8268, groups:  ID, 318

Fixed effects:
                        Estimate Std. Error z value Pr(>|z|)    
(Intercept)             -7.46295    0.57105 -13.069  < 2e-16 ***
mail_cent                1.52215    0.28451   5.350 8.80e-08 ***
critical_cent           -0.01644    0.38082  -0.043    0.966    
mail_cent:critical_cent  2.46177    0.54242   4.539 5.67e-06 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr) ml_cnt cr

,2.5 %,97.5 %
(Intercept),-8.5821812,-6.3437256
mail_cent,0.9645156,2.0797905
critical_cent,-0.7628417,0.7299531
mail_cent:critical_cent,1.3986507,3.5248957


effect,term,estimate,std.error,statistic,p.value,conf.low,conf.high
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
fixed,(Intercept),5.739585e-04,0.0003277562,-13.06893873,4.955200e-39,0.0001874157,0.001757741
fixed,mail_cent,4.582080e+00,1.3036664939,5.35000860,8.795005e-08,2.6235164965,8.002792029
fixed,critical_cent,9.836902e-01,0.3746108744,-0.04318104,9.655572e-01,0.4663393366,2.074983373
fixed,mail_cent:critical_cent,1.172558e+01,6.3601845696,4.53850494,5.665447e-06,4.0497318034,33.950231156


### Dependant variable:  kWh

In [7]:
m2 <- lmer(charge ~ mail_cent*critical_cent+(1|ID),data=df_maildays)
summary(m2)
confint(m2,parm="beta_",method="Wald")
tidy(m2,conf.int=TRUE,exponentiate=TRUE,effects="fixed")

Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: charge ~ mail_cent * critical_cent + (1 | ID)
   Data: df_maildays

REML criterion at convergence: 29370

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.3239 -0.2412  0.0067  0.0540 25.5930 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID       (Intercept) 0.179    0.4231  
 Residual             1.946    1.3948  
Number of obs: 8268, groups:  ID, 318

Fixed effects:
                         Estimate Std. Error        df t value Pr(>|t|)    
(Intercept)             7.034e-02  3.017e-02 4.094e+02   2.331   0.0202 *  
mail_cent               2.602e-01  3.152e-02 8.211e+03   8.255  < 2e-16 ***
critical_cent           3.073e-02  3.697e-02 7.947e+03   0.831   0.4060    
mail_cent:critical_cent 3.877e-01  6.138e-02 7.947e+03   6.316 2.83e-10 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr) ml_cnt crt

,2.5 %,97.5 %
(Intercept),0.01120169,0.1294822
mail_cent,0.19843829,0.3220051
critical_cent,-0.04174295,0.1031961
mail_cent:critical_cent,0.26735456,0.5079617


effect,term,estimate,std.error,statistic,df,p.value,conf.low,conf.high
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
fixed,(Intercept),1.072875,0.03237311,2.3311984,409.3851,2.022755e-02,1.0112647,1.138239
fixed,mail_cent,1.297218,0.04089182,8.2550533,8211.4038,1.754762e-16,1.2194968,1.379892
fixed,critical_cent,1.031204,0.03812869,0.8310114,7946.9230,4.059922e-01,0.9591163,1.108709
fixed,mail_cent:critical_cent,1.473526,0.09044576,6.3156560,7946.9230,2.834794e-10,1.3065036,1.661900


## Model 2 – Weekday and no-event day addition to regression model
### Dependant variable:  Number of charges

In [8]:
m1 <- glmer(charge_started ~ mail_cent*critical_cent*weekday_cent+(1|ID),data=df_maildays,family = binomial(link = "logit"))
summary(m1)
confint(m1,parm="beta_",method="Wald")
tidy(m1,conf.int=TRUE,exponentiate=TRUE,effects="fixed")

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.273994 (tol = 0.001, component 1)”


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: charge_started ~ mail_cent * critical_cent * weekday_cent + (1 |  
    ID)
   Data: df_maildays

     AIC      BIC   logLik deviance df.resid 
  1004.0   1067.2   -493.0    986.0     8259 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.2146 -0.0759 -0.0253 -0.0198 15.4711 

Random effects:
 Groups Name        Variance Std.Dev.
 ID     (Intercept) 5.754    2.399   
Number of obs: 8268, groups:  ID, 318

Fixed effects:
                                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)                          -7.37816    0.52582 -14.032  < 2e-16 ***
mail_cent                             1.53001    0.30228   5.062 4.16e-07 ***
critical_cent                         0.05973    0.41092   0.145 0.884427    
weekday_cent                         -0.23930    0.38180  -0.627 0.530812    
mail_cent:critical_cent               2.

,2.5 %,97.5 %
(Intercept),-8.4087587,-6.3475687
mail_cent,0.9375467,2.1224675
critical_cent,-0.7456581,0.8651202
weekday_cent,-0.9876121,0.5090139
mail_cent:critical_cent,1.1122171,3.3769727
mail_cent:weekday_cent,-1.0107513,1.1251623
critical_cent:weekday_cent,-1.3462969,1.6411990
mail_cent:critical_cent:weekday_cent,-2.9979221,1.2367254


effect,term,estimate,std.error,statistic,p.value,conf.low,conf.high
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
fixed,(Intercept),0.0006247471,0.0003285067,-14.0316370,9.982048e-45,0.0002229064,0.001750999
fixed,mail_cent,4.6182096136,1.3959982937,5.0615345,4.158955e-07,2.5537086797,8.351720071
fixed,critical_cent,1.0615509889,0.4362129291,0.1453591,8.844273e-01,0.4744219814,2.375291505
fixed,weekday_cent,0.7871793763,0.3005445896,-0.6267667,5.308122e-01,0.3724650221,1.663649829
fixed,mail_cent:critical_cent,9.4365920085,5.4520322717,3.8850332,1.023159e-04,3.0410934603,29.281990145
fixed,mail_cent:weekday_cent,1.0588734155,0.5769652330,0.1049862,9.163867e-01,0.3639454502,3.080716930
fixed,critical_cent:weekday_cent,1.1588765890,0.8832149573,0.1934723,8.465891e-01,0.2602020453,5.161354313
fixed,mail_cent:critical_cent:weekday_cent,0.4145347862,0.4478165701,-0.8151522,4.149852e-01,0.0498906266,3.444316110


### Dependant variable:  kWh

In [9]:
m2 <- lmer(charge ~ mail_cent*critical_cent*weekday_cent+(1|ID),data=df_maildays)
summary(m2)
confint(m2,parm="beta_",method="Wald")
tidy(m2,conf.int=TRUE,exponentiate=TRUE,effects="fixed")


Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: charge ~ mail_cent * critical_cent * weekday_cent + (1 | ID)
   Data: df_maildays

REML criterion at convergence: 29368

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.4257 -0.1737  0.0104  0.0514 25.5103 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID       (Intercept) 0.1791   0.4232  
 Residual             1.9424   1.3937  
Number of obs: 8268, groups:  ID, 318

Fixed effects:
                                       Estimate Std. Error         df t value
(Intercept)                             0.06518    0.03076  441.52004   2.119
mail_cent                               0.24581    0.03294 8210.19974   7.461
critical_cent                           0.02898    0.03880 7942.96902   0.747
weekday_cent                           -0.03421    0.03733 8050.95044  -0.917
mail_cent:critical_cent                 0.34487    0.06411 7942.96902   5.379
mail_cent:weekday_cent  

,2.5 %,97.5 %
(Intercept),0.004885243,0.12546513
mail_cent,0.181237789,0.31037312
critical_cent,-0.047065722,0.10501732
weekday_cent,-0.107381167,0.03895269
mail_cent:critical_cent,0.219209392,0.47052372
mail_cent:weekday_cent,-0.226849047,0.02090217
critical_cent:weekday_cent,-0.160377155,0.12981291
mail_cent:critical_cent:weekday_cent,-0.536770398,-0.05440560


effect,term,estimate,std.error,statistic,df,p.value,conf.low,conf.high
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
fixed,(Intercept),1.0673460,0.03283235,2.1187782,441.520,3.466702e-02,1.0048972,1.1336756
fixed,mail_cent,1.2786508,0.04212297,7.4614722,8210.200,9.430539e-14,1.1987002,1.3639339
fixed,critical_cent,1.0293997,0.03993804,0.7468488,7942.969,4.551769e-01,0.9540247,1.1107298
fixed,weekday_cent,0.9663644,0.03607511,-0.9165163,8050.950,3.594236e-01,0.8981833,1.0397213
fixed,mail_cent:critical_cent,1.4118015,0.09051338,5.3791285,7942.969,7.697885e-08,1.2450920,1.6008324
fixed,mail_cent:weekday_cent,0.9021509,0.05701865,-1.6292492,8207.653,1.032986e-01,0.7970411,1.0211221
fixed,critical_cent:weekday_cent,0.9848341,0.07290671,-0.2064330,7942.969,8.364580e-01,0.8518225,1.1386153
fixed,mail_cent:critical_cent:weekday_cent,0.7440939,0.09156411,-2.4020900,7942.969,1.632451e-02,0.5846333,0.9470479


## Model 3 – Comparison of only non-critical times for event days and no-event days
### Dependant variable:  Number of charges

In [10]:
df_other <- subset(df, df$critical == 0)
m1 <- glmer(charge_started ~ mail_cent*mailday_cent +(1|ID),data=df_other,family = binomial(link = "logit"))
summary(m1)
confint(m1,parm="beta_",method="Wald")
tidy(m1,conf.int=TRUE,exponentiate=TRUE,effects="fixed")


fixed-effect model matrix is rank deficient so dropping 1 column / coefficient



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: charge_started ~ mail_cent * mailday_cent + (1 | ID)
   Data: df_other

     AIC      BIC   logLik deviance df.resid 
   974.1   1004.1   -483.1    966.1    13352 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-0.5947 -0.0157 -0.0157 -0.0157  9.0122 

Random effects:
 Groups Name        Variance Std.Dev.
 ID     (Intercept) 12.66    3.558   
Number of obs: 13356, groups:  ID, 318

Fixed effects:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)   -8.2370     0.8042 -10.242   <2e-16 ***
mail_cent      0.3259     0.4073   0.800    0.424    
mailday_cent  -0.3228     0.3171  -1.018    0.309    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr) ml_cnt
mail_cent   -0.029       
mailday_cnt  0.029 -0.664
fit warnings:
fixed-effect model matrix is rank deficient so d

,2.5 %,97.5 %
(Intercept),-9.8132569,-6.6608310
mail_cent,-0.4724851,1.1241945
mailday_cent,-0.9444048,0.2987418


effect,term,estimate,std.error,statistic,p.value,conf.low,conf.high
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
fixed,(Intercept),0.0002646655,0.0002128453,-10.2424673,1.279316e-24,5.472133e-05,0.001280082
fixed,mail_cent,1.3852140870,0.5642305572,0.7999895,4.237169e-01,6.234510e-01,3.077736818
fixed,mailday_cent,0.7240958562,0.2296361906,-1.0179621,3.086959e-01,3.889110e-01,1.348161510


### Dependant variable:  kWh

In [11]:
m2 <- lmer(charge ~ mail_cent* mailday_cent + (1|ID),data=df_other)
summary(m2)
confint(m2,parm="beta_",method="Wald")
tidy(m2,conf.int=TRUE,exponentiate=TRUE,effects="fixed")

fixed-effect model matrix is rank deficient so dropping 1 column / coefficient



Linear mixed model fit by REML. t-tests use Satterthwaite's method [
lmerModLmerTest]
Formula: charge ~ mail_cent * mailday_cent + (1 | ID)
   Data: df_other

REML criterion at convergence: 38551.4

Scaled residuals: 
   Min     1Q Median     3Q    Max 
-2.588 -0.049 -0.023 -0.021 44.076 

Random effects:
 Groups   Name        Variance Std.Dev.
 ID       (Intercept) 0.06744  0.2597  
 Residual             1.01590  1.0079  
Number of obs: 13356, groups:  ID, 318

Fixed effects:
               Estimate Std. Error         df t value Pr(>|t|)    
(Intercept)   8.134e-02  1.697e-02  3.169e+02   4.792 2.54e-06 ***
mail_cent     6.687e-02  3.163e-02  1.318e+04   2.114   0.0345 *  
mailday_cent -3.842e-02  2.435e-02  1.310e+04  -1.578   0.1147    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Correlation of Fixed Effects:
            (Intr) ml_cnt
mail_cent    0.000       
mailday_cnt  0.000 -0.632
fit warnings:
fixed-effect model matrix is rank deficient so dropping 1 co

,2.5 %,97.5 %
(Intercept),0.048074369,0.114614843
mail_cent,0.004878462,0.128867892
mailday_cent,-0.086152587,0.009314264


effect,term,estimate,std.error,statistic,df,p.value,conf.low,conf.high
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
fixed,(Intercept),1.0847446,0.01841346,4.792046,316.9122,2.541208e-06,1.0492487,1.121441
fixed,mail_cent,1.0691599,0.03381810,2.114197,13176.8916,3.451721e-02,1.0048904,1.137540
fixed,mailday_cent,0.9623095,0.02343631,-1.577515,13096.9594,1.147013e-01,0.9174542,1.009358
